## Import

In [ ]:
import random
import os
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from sklearn.preprocessing import LabelEncoder

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from numba import cuda
import os

In [ ]:
SKIP_COUNT = 10

In [ ]:
print(torch.__version__)
print(torch.cuda.is_available())    # cuda 사용 가능 여부 확인
print(torch.cuda.device_count())    # 사용 가능한 GPU 개수 확인
device = cuda.get_current_device()  # 현재 사용하고 있는 GPU 확인
#device.reset()                     # GPU 캐시 리셋
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'    # GPU out of memory error 발생 시 주석 처리
os.environ['CUDA_VISIBLE_DEVICES'] = '0'    # 사용할 GPU 번호 설정
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')     # cuda 사용 가능 여부에 따라 device 정보 저장

## Hyperparameter Setting

In [ ]:
CFG = {
    'TRAIN_WINDOW_SIZE':90, # 90일치로 학습
    'PREDICT_SIZE':21, # 21일치 예측
    'EPOCHS':15,    # 학습횟수
    'LEARNING_RATE':1e-4,
    'BATCH_SIZE':256,
    'SEED':41   # 시드 고정
}

In [ ]:
def seed_everything(seed):  # Seed 고정 함수
    random.seed(seed)      # random seed 고정
    os.environ['PYTHONHASHSEED'] = str(seed)    # PYTHONHASHSEED 값 설정
    np.random.seed(seed)    # numpy seed 고정
    torch.manual_seed(seed) # torch seed 고정
    torch.cuda.manual_seed(seed)    # torch cuda seed 고정
    torch.backends.cudnn.deterministic = True   # torch cudnn seed 고정
    torch.backends.cudnn.benchmark = True   # cudnn을 빠르게 하기 위한 옵션으로, 연산 진행시 어떤 알고리즘을 쓸지를 정하는 부분이다.

seed_everything(CFG['SEED']) # Seed 고정 함수실행

### 데이터 불러오기

In [ ]:
train_data_sales = pd.read_csv('./data/train.csv').drop(columns=['ID', '제품'])    # ID, 제품 컬럼 삭제
train_data_sales = train_data_sales[::SKIP_COUNT]
train_data_sales

In [ ]:
train_data_brand_tmp = pd.read_csv('./data/brand_keyword_cnt.csv')
sales_category = train_data_sales[['대분류', '중분류', '소분류', '브랜드']]
train_data_brand = pd.merge(sales_category, train_data_brand_tmp, on = '브랜드', how='inner')
train_data_brand

In [ ]:
print(f'train data brand null count: {train_data_brand.isnull().sum()}')
print(f'null index : {train_data_brand[train_data_brand["2022-01-01"].isnull()].index}')
train_data_brand.fillna(train_data_brand["2022-01-01"].mean(), inplace=True)
print(f'train data brand null count: {train_data_brand.isnull().sum()}')
print(f'null index : {train_data_brand[train_data_brand["2022-01-01"].isnull()].index}')

### 데이터 전처리

In [ ]:
def scale_minmax(dataframe):
    numeric_cols = dataframe.columns[4:]
    # 칵 column의 min 및 max 계산
    min_values = dataframe[numeric_cols].min(axis=1)
    max_values = dataframe[numeric_cols].max(axis=1)
    # 각 행의 범위(max-min)를 계산하고, 범위가 0인 경우 1로 대체
    ranges = max_values - min_values
    ranges[ranges == 0] = 1
    # min-max scaling 수행
    dataframe[numeric_cols] = (dataframe[numeric_cols].subtract(min_values, axis=0)).div(ranges, axis=0)
    # max와 min 값을 dictionary 형태로 저장
    scale_min_dict = min_values.to_dict()
    scale_max_dict = max_values.to_dict()
    
    return scale_min_dict, scale_max_dict


scale_min_dict_sales, scale_max_dict_sales = scale_minmax(train_data_sales)
scale_min_dict_brand, scale_max_dict_brand = scale_minmax(train_data_brand)

In [ ]:
# Label Encoding 문자형 변수를 숫자로 인코딩
label_encoder = LabelEncoder()  # sklearn.preprocessing 패키지의 LabelEncoder() 함수를 사용하여 라벨 인코더 생성
categorical_columns = ['대분류', '중분류', '소분류', '브랜드']  # 문자형 변수를 숫자로 인코딩할 변수명 리스트

for col in categorical_columns:
    label_encoder.fit(train_data_sales[col])  # fit 함수를 통해 라벨 인코더 학습
    train_data_sales[col] = label_encoder.transform(train_data_brand[col])  # transform 함수를 통해 라벨 인코딩 수행

for col in categorical_columns:
    label_encoder.fit(train_data_brand[col])
    train_data_brand[col] = label_encoder.transform(train_data_brand[col])

In [ ]:
def make_train_data(data, train_size=CFG['TRAIN_WINDOW_SIZE'], predict_size=CFG['PREDICT_SIZE']):
    '''
    학습 기간 블럭, 예측 기간 블럭의 세트로 데이터를 생성
    data : 일별 판매량
    train_size : 학습에 활용할 기간
    predict_size : 추론할 기간
    '''
    num_rows = len(data)    # 제품의 개수
    window_size = train_size + predict_size   # 학습 기간 + 예측 기간으로 윈도우사이즈 결정
    
    #빈 배열 생성
    input_data = np.empty((num_rows * (len(data.columns) - window_size + 1), train_size, len(data.iloc[0, :4]) + 1))   
    target_data = np.empty((num_rows * (len(data.columns) - window_size + 1), predict_size))   
    
    for i in tqdm(range(num_rows)):
        encode_info = np.array(data.iloc[i, :4])    # 해당 제품의 앞쪽 4개 열 데이터 [대분류, 중분류, 소분류, 브랜드] encode_info로 저장
        brand_data = np.array(data.iloc[i, 4:])   # 실제 판매량 정보
        
        for j in range(len(brand_data) - window_size + 1):
            window = brand_data[j : j + window_size]    # 현재 조합에 해당하는 윈도우 추출
            temp_data = np.column_stack((np.tile(encode_info, (train_size, 1)), window[:train_size]))   # encode_info를 train_size만큼 반복하여 학습 데이터와 결합한 뒤, 윈도우의 앞쪽 train_size 개의 데이터와 함께 temp_data로 저장
            input_data[i * (len(data.columns) - window_size + 1) + j] = temp_data   # input_data에 temp_data 저장
            target_data[i * (len(data.columns) - window_size + 1) + j] = window[train_size:]    # target_data에 윈도우의 뒤쪽 predict_size 개의 데이터 저장
    
    return input_data, target_data

In [ ]:
def make_predict_data(data, train_size=CFG['TRAIN_WINDOW_SIZE']):
    '''
    평가 데이터(Test Dataset)를 추론하기 위한 Input 데이터를 생성
    data : 일별 판매량
    train_size : 추론을 위해 필요한 일별 판매량 기간 (= 학습에 활용할 기간)
    '''
    num_rows = len(data)    # 제품의 개수 ( =행의 개수)
    
    input_data = np.empty((num_rows, train_size, len(data.iloc[0, :4]) + 1))    # 빈 배열 생성
    
    for i in tqdm(range(num_rows)):   
        encode_info = np.array(data.iloc[i, :4])    # 해당 제품의 앞쪽 4개 열 데이터 [대분류, 중분류, 소분류, 브랜드] encode_info로 저장
        sales_data = np.array(data.iloc[i, -train_size:])   # 실제 판매량 정보
        
        window = sales_data[-train_size : ]   # 추론을 위한 일별 판매량 기간만큼의 데이터를 window로 저장
        temp_data = np.column_stack((np.tile(encode_info, (train_size, 1)), window[:train_size]))   # encode_info를 train_size만큼 반복하고, 추론 기간에 해당하는 데이터와 함께 열 방향으로 합쳐서 temp_data를 생성
        input_data[i] = temp_data
    
    return input_data

In [ ]:
train_input_sales, train_target_sales = make_train_data(train_data_sales)
test_input_sales = make_predict_data(train_data_sales)

train_input_brand, train_target_brand = make_train_data(train_data_brand)
test_input_brand = make_predict_data(train_data_brand)

In [ ]:
# Train / Validation Split (고정)
# 맨 뒤 데이터의 20%를 Validation Set으로 사용
data_len_sales = len(train_input_sales)
val_input_sales = train_input_sales[-int(data_len_sales*0.2):]
val_target_sales = train_target_sales[-int(data_len_sales*0.2):]
train_input_sales = train_input_sales[:-int(data_len_sales*0.2)]
train_target_sales = train_target_sales[:-int(data_len_sales*0.2)]

data_len_brand = len(train_input_brand)
val_input_brand = train_input_brand[-int(data_len_brand*0.2):]
val_target_brand = train_target_brand[-int(data_len_brand*0.2):]
train_input_brand = train_input_brand[:-int(data_len_brand*0.2)]
train_target_brand = train_target_brand[:-int(data_len_brand*0.2)]

In [ ]:
print(train_input_sales.shape, train_target_sales.shape, val_input_sales.shape, val_target_sales.shape, test_input_sales.shape)
print(train_input_brand.shape, train_target_brand.shape, val_input_brand.shape, val_target_brand.shape, test_input_brand.shape)

### Custom Dataset

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y
        
    def __getitem__(self, index):
        if self.Y is not None:
            return torch.Tensor(self.X[index]), torch.Tensor(self.Y[index])
        return torch.Tensor(self.X[index])
    
    def __len__(self):
        return len(self.X)

In [ ]:
'''
Dataset 은 데이터셋의 특징(feature)을 가져오고 하나의 샘플에 정답(label)을 지정하는 일을 한 번에 합니다.
모델을 학습할 때, 일반적으로 샘플들을 《미니배치(minibatch)》로 전달하고, 매 에폭(epoch)마다 데이터를 다시 섞어서 과적합(overfit)을 막고,
Python의 multiprocessing 을 사용하여 데이터 검색 속도를 높이려고 합니다.
DataLoader 는 간단한 API로 이러한 복잡한 과정들을 추상화한 순회 가능한 객체(iterable)입니다.
'''

train_dataset_sales = CustomDataset(train_input_sales, train_target_sales)
train_loader_sales = DataLoader(train_dataset_sales, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=0)

val_dataset_sales = CustomDataset(val_input_sales, val_target_sales)
val_loader_sales = DataLoader(val_dataset_sales, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

train_dataset_brand = CustomDataset(train_input_brand, train_target_brand)
train_loader_brand = DataLoader(train_dataset_brand, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=0)

val_dataset_brand = CustomDataset(val_input_brand, val_target_brand)
val_loader_brand = DataLoader(val_dataset_brand, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

### 모델 선언

In [ ]:
class MultiModalModel(nn.Module):
    def __init__(self, input_size_brand=5, input_size_sales=5, hidden_size=512, num_layers=4, output_size=CFG["PREDICT_SIZE"]):
        super(MultiModalModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.lstm_brand = nn.LSTM(input_size_brand, hidden_size, num_layers = self.num_layers, batch_first=True)
        self.lstm_sales = nn.LSTM(input_size_sales, hidden_size, num_layers = self.num_layers, batch_first=True)

        # Fully connected layer for prediction
        self.fc = nn.Sequential(
            nn.Linear(hidden_size * 2, hidden_size),  # Concatenated hidden states from brand and sales
            nn.Mish(),                                
            nn.Dropout(0.5),
            nn.Linear(hidden_size, output_size)
        )
        self.actv = nn.Mish()

    def forward(self, x_brand, x_sales):
        batch_size = x_brand.size(0)

        # Brand LSTM
        hidden_brand = self.init_hidden(batch_size, x_brand.device)
        lstm_out_brand, _ = self.lstm_brand(x_brand, hidden_brand)
        last_output_brand = lstm_out_brand[:, -1, :]

        # Sales LSTM
        hidden_sales = self.init_hidden(batch_size, x_sales.device)
        lstm_out_sales, _ = self.lstm_sales(x_sales, hidden_sales)
        last_output_sales = lstm_out_sales[:, -1, :]

        # Concatenate brand and sales outputs
        fused_output = torch.cat((last_output_brand, last_output_sales), dim=1)

        # Fully connected layer
        output = self.actv(self.fc(fused_output))
        return output.squeeze(1)

    def init_hidden(self, batch_size, device):
        return (torch.zeros(self.num_layers, batch_size, self.hidden_size, device=device),
                torch.zeros(self.num_layers, batch_size, self.hidden_size, device=device))


### 모델 학습

In [ ]:
def validation(model, val_loader_brand, val_loader_sales, criterion, device):
    model.eval()
    val_loss = []
    
    with torch.no_grad():   # 검증 단계이므로 역전파가 필요없음, no_grad()를 통해 메모리 사용량을 줄임
        for (X_brand, _), (X_sales, Y_sales) in zip(tqdm(val_loader_brand), val_loader_sales):
            X_brand = X_brand.to(device)
            X_sales = X_sales.to(device)
            Y_sales = Y_sales.to(device)
            
            output = model(X_brand, X_sales)  # 입력으로 브랜드 키워드 카운트와 판매 데이터를 함께 사용
            loss = criterion(output, Y_sales)
            
            val_loss.append(loss.item())
    return np.mean(val_loss)

In [ ]:
def train(model, optimizer, train_loader_brand, train_loader_sales, val_loader_brand, val_loader_sales, device):
    model.to(device)
    criterion = nn.MSELoss().to(device)
    best_loss = float('inf')
    best_model = None
    
    for epoch in range(1, CFG['EPOCHS'] + 1):
        model.train()
        train_loss = []
        
        for (X_brand, _), (X_sales, Y_sales) in zip(tqdm(train_loader_brand), train_loader_sales):
            X_brand = X_brand.to(device)
            X_sales = X_sales.to(device)
            Y_sales = Y_sales.to(device)
            
            optimizer.zero_grad()
            
            output = model(X_brand, X_sales)  # 입력으로 브랜드 키워드 카운트와 판매 데이터를 함께 사용
            loss = criterion(output, Y_sales)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
            
        val_loss = validation(model, val_loader_brand, val_loader_sales, criterion, device)
        print(f'Epoch [{epoch}/{CFG["EPOCHS"]}] - Train Loss: {np.mean(train_loss):.5f} - Val Loss: {val_loss:.5f}')
        
        if val_loss < best_loss:
            best_loss = val_loss
            best_model = model
            print('Best model updated')
    
    return best_model


## Run !!

In [ ]:
model = MultiModalModel()
optimizer = torch.optim.NAdam(params = model.parameters(), lr = CFG["LEARNING_RATE"])    # Adam optimizer를 사용
infer_model = train(model, optimizer, train_loader_brand, train_loader_sales, val_loader_brand, val_loader_sales, device)   # 학습 진행

In [ ]:
def loadModel(model_path):
    loaded_model = MultiModalModel()
    loaded_model.load_state_dict(torch.load(model_path))
    return loaded_model

# model = loadModel('./model/MultiModal.pt')

In [ ]:
def saveModel(model, PATH):
    torch.save(model.state_dict(), PATH)

saveModel(infer_model, './model/MultiModal_NAdam_Mish.pt')

## 모델 추론

In [ ]:
test_dataset_sales = CustomDataset(test_input_sales, None)
test_loader_sales = DataLoader(test_dataset_sales, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

test_dataset_brand = CustomDataset(test_input_brand, None)
test_loader_brand = DataLoader(test_dataset_brand, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [ ]:
def inference(model, test_loader_brand, test_loader_sales, device):
    predictions = []
    
    with torch.no_grad():        
        for X_brand, X_sales in zip(tqdm(test_loader_brand), test_loader_sales):
            X_brand = X_brand.to(device)
            X_sales = X_sales.to(device)
            
            output = model(X_brand, X_sales)
            output = output.cpu().numpy()
            predictions.extend(output)
    
    return np.array(predictions)

In [ ]:
pred = inference(infer_model, test_loader_brand, test_loader_sales, device)

In [ ]:
# 추론 결과를 inverse scaling
for idx in range(len(pred)):
    pred[idx, :] = pred[idx, :] * (scale_max_dict_sales[idx * SKIP_COUNT] - scale_min_dict_sales[idx* SKIP_COUNT]) + scale_min_dict_sales[idx* SKIP_COUNT]

pred = np.round(pred, 0).astype(int)

In [ ]:
pred.shape

## Submission

In [ ]:
submit = pd.read_csv('./data/sample_submission.csv')
submit

In [ ]:
submit.iloc[:,1:] = pred
submit.to_csv('./output/MultiModal_submit.csv', index=False)

In [ ]:
submit